In [ ]:
%%capture
! pip install pip3-autoremove
! pip-autoremove torch torchvision torchaudio -y
! pip install torch xformers triton
# ! pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
! pip install unsloth

In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [ ]:
BASE_PATH = '..'

In [2]:
import os

print(os.getenv('CUDA_VISIBLE_DEVICES'))

0


In [ ]:
import sys
sys.path.append(BASE_PATH)

In [4]:
import json

import torch
import numpy as np

from datasets import DatasetDict, Dataset

from unsloth import FastLanguageModel

from tqdm.auto import tqdm

from logger import get_logger


/home/stepan/.conda/envs/llm-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [5]:
# ! pip install --no-deps --upgrade "flash-attn>=2.6.3"

In [6]:
EVAL_BATCH_SIZE = 1
MAX_SEQ_LENGTH = 1024

In [7]:
log = get_logger(f'logs/gemma-2-2b', 'arc-agi')

In [8]:
device_id = 0

In [9]:
def get_model_tokenizer(max_seq_length=1024, dtype=None, load_in_4bit=True, add_lora=False):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/gemma-2-2b-it",
        # model_name="models/gemma-2-2b/checkpoint-1562",
        max_seq_length=max_seq_length,
        dtype=dtype,
        load_in_4bit=load_in_4bit,
        device_map={'': 0},
        # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

    if add_lora:
        model = FastLanguageModel.get_peft_model(
            model,
            r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                            "gate_proj", "up_proj", "down_proj", ],
            lora_alpha=16,
            lora_dropout=0,  # Supports any, but = 0 is optimized
            bias="none",  # Supports any, but = "none" is optimized
            # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
            use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
            random_state=3407,
            use_rslora=False,  # We support rank stabilized LoRA
            loftq_config=None,  # And LoftQ
        )

    tokenizer.truncation_side = 'left'
    tokenizer.padding_side = 'left'

    return model, tokenizer

In [10]:
def eval(f):
    def wrapper(model, *args, **kwargs):
        FastLanguageModel.for_inference(model)
        return f(model, *args, **kwargs)

    return wrapper


def train(f):
    def wrapper(model, *args, **kwargs):
        FastLanguageModel.for_training(model)
        return f(model, *args, **kwargs)

    return wrapper

In [11]:
model, tokenizer = get_model_tokenizer(max_seq_length=MAX_SEQ_LENGTH, add_lora=True)

Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.43.4.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.677 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [12]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

In [13]:
def prepare_dataset(tokenizer, device, seed):
    # 3 classes
    # Load data from JSON files
    def load_data(file_path):
        with open(file_path, 'r') as f:
            data = json.load(f)
        return data
    
    # Load all datasets
    training_challenges = load_data(f'{BASE_PATH}/arc-agi_training_challenges.json')
    training_solutions = load_data(f'{BASE_PATH}/arc-agi_training_solutions.json')
    evaluation_challenges = load_data(f'{BASE_PATH}/arc-agi_evaluation_challenges.json')
    evaluation_solutions = load_data(f'{BASE_PATH}/arc-agi_evaluation_solutions.json')
    test_challenges = load_data(f'{BASE_PATH}/arc-agi_test_challenges.json')
    
    train_dataset = Dataset.from_dict(training_challenges)

    PROMPT = '''### Tweet:
    {}
    
    ### Classification:
    {}'''

    def formatting_prompts_func(examples):
        inputs = examples["tweet"]
        outputs = [
            HATE_TOKEN if label in [1, 0] else NORMAL_TOKEN for label in examples["class"]
        ]
        texts = []
        prompts = []
        for input, output in zip(inputs, outputs):
            text = PROMPT.format(input.strip(), output) + EOS_TOKEN
            texts.append(text)
            prompts.append(PROMPT.format(input, ""))
        return {"ref": texts, 'prompt': prompts}

    dataset = dataset.map(formatting_prompts_func, batched=True)

    # Split the training set into training (80%) and validation (20%) sets
    train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
    test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)

    # Assign datasets
    train_dataset = train_testvalid['train']
    val_dataset = test_valid['test']
    test_dataset = test_valid['train']

    dataset = DatasetDict({
        'train': train_dataset,
        'test': test_dataset,
        'val': val_dataset
    })
    return dataset.rename_columns({'class': 'label'})


In [14]:
dataset = prepare_dataset(tokenizer, f"cuda:{device_id}", seed=3407)
dataset

DatasetDict({
    train: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'label', 'tweet', 'ref', 'prompt'],
        num_rows: 19826
    })
    test: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'label', 'tweet', 'ref', 'prompt'],
        num_rows: 2478
    })
    val: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'label', 'tweet', 'ref', 'prompt'],
        num_rows: 2479
    })
})

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        dataset['test']['prompt'][0],
    ], return_tensors="pt").to(f"cuda:{device_id}")

outputs = model.generate(**inputs, max_new_tokens=1)
res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [17]:
model.device

device(type='cuda', index=0)

In [18]:
def gpu_stats(device_id=0):
    #@title Show current memory stats
    gpu_stats = torch.cuda.get_device_properties(device_id)
    start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
    max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
    return {'gpu': gpu_stats.name, 'max_memory': max_memory, 'start_gpu_memory': start_gpu_memory}

In [19]:
TOKENS = {
    'Positive': ...,
    'Negative': ...,

    'Hate': ...,
    'Normal': ...,
    'Fake': ...,
    'Truth': ...
}

In [20]:
for key, val in TOKENS.items():
    code = tokenizer.encode(key, add_special_tokens=False)
    print(f"{key}: {code}")
    TOKENS[key] = code[0]

Positive: [35202]
Negative: [39654]
Hate: [88060]
Normal: [15273]
Fake: [41181]
Truth: [55882]


In [21]:
def softmax(x, axis=None):
    # Subtract the max for numerical stability
    x_max = np.max(x, axis=axis, keepdims=True)
    x_stable = x - x_max

    # Compute the exponentials
    exp_x = np.exp(x_stable)

    # Compute the softmax
    sum_exp_x = np.sum(exp_x, axis=axis, keepdims=True)
    softmax_x = exp_x / sum_exp_x

    return softmax_x

In [22]:
@eval
def evaluate(model, tokenizer, dataset, batch_size, threshold=0.5):
    eval_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

    preds = []
    labels = []
    for i, batch in tqdm(enumerate(eval_dataloader), total=len(eval_dataloader)):
        texts = batch["prompt"]

        inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)

        outputs = model.generate(
            **inputs,
            max_new_tokens=1,
            output_scores=True,
            return_dict_in_generate=True
        )  # (B, len, vocab_size)

        pos = outputs.scores[0][:, TOKENS['Hate']].cpu().float().numpy()
        neg = outputs.scores[0][:, TOKENS['Normal']].cpu().float().numpy()

        preds.extend(np.array([neg, pos]).T)
        labels.extend(batch["label"])

        if (i + 1) % 1000 == 0:
            log.warn(f'GPU Stats: {gpu_stats(device_id)}')

    preds = softmax(np.array(preds), axis=-1)[:, 1] > threshold

    val_acc = accuracy_score(labels, preds)

    log.info(f"Accuracy: {val_acc}")
    return val_acc, preds, labels

In [23]:
eval_acc, preds, labels = evaluate(model, tokenizer, dataset["test"], batch_size=EVAL_BATCH_SIZE)